In [55]:
// https://spark.apache.org/docs/3.3.0/api/scala/org/apache/spark/sql/Dataset.html
// https://spark.apache.org/docs/latest/sql-data-sources-csv.html
// val columns = Seq("StudentID", "Course")
// val data = Seq(("1", "Spark"), ("2", "Scala"), ("3", "Java"))
// val fromRDD: DataFrame = spark.sparkContext.parallelize(data).toDF(columns: _*)
// fromRDD.show()

In [2]:
import $ivy.`org.apache.spark::spark-sql:3.3.0`

import $ivy.$

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [18]:
val spark = (SparkSession.builder()
             .master("local[*]")
             .appName("OTUS lesson")
             .getOrCreate()) 
spark.sparkContext.setLogLevel("ERROR")

spark: SparkSession = org.apache.spark.sql.SparkSession@4736f7da

In [19]:
import spark.implicits._

import spark.implicits._

In [21]:
println(s"spark.version == ${spark.version}")

spark.version == 3.3.0


In [24]:
val tripsDF = spark.read.parquet("yellow_taxi.parquet")
val zonesDF = spark.read.option("header", "true").csv("taxi_zones.csv")

tripsDF: DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp ... 15 more fields]
zonesDF: DataFrame = [LocationID: string, Borough: string ... 2 more fields]

In [25]:
tripsDF.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2| 2018-01-25 02:02:56|  2018-01-25 02:10:58|              1|         2.02|         1|                 N|          48|         107|           2|        8.5|  0.5|    0.5|       0.0|         0.0|                  0.3|         9.8|
|       2| 2018-01-25 02:57:13|  2018-01-25 03:2

In [16]:
ridesDF.dtypes

res16: Array[(String, String)] = Array(
  ("VendorID", "IntegerType"),
  ("tpep_pickup_datetime", "TimestampType"),
  ("tpep_dropoff_datetime", "TimestampType"),
  ("passenger_count", "IntegerType"),
  ("trip_distance", "DoubleType"),
  ("RatecodeID", "IntegerType"),
  ("store_and_fwd_flag", "StringType"),
  ("PULocationID", "IntegerType"),
  ("DOLocationID", "IntegerType"),
  ("payment_type", "IntegerType"),
  ("fare_amount", "DoubleType"),
  ("extra", "DoubleType"),
  ("mta_tax", "DoubleType"),
  ("tip_amount", "DoubleType"),
  ("tolls_amount", "DoubleType"),
  ("improvement_surcharge", "DoubleType"),
  ("total_amount", "DoubleType")
)

In [26]:
zonesDF.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|        BORO|
|         3|        Bronx|Allerton/Pelham G...|        BORO|
|         4|    Manhattan|       Alphabet City|      yeLLow|
|         5|Staten Island|       Arden Heights|        BORO|
|         6|Staten Island|Arrochar/Fort Wad...|        BORO|
|         7|       Queens|             Astoria|        bOrO|
|         8|       Queens|        Astoria Park|        BORO|
|         9|       Queens|          Auburndale|        BORO|
|        10|       Queens|        Baisley Park|        BORO|
|        11|     Brooklyn|          Bath Beach|        Boro|
|        12|    Manhattan|        Battery Park|      YELLOW|
|        13|    Manhattan|   Battery Park City|      yeLLow|
|        14|     Brookly

In [27]:
tripsDF.createOrReplaceTempView("trips")
zonesDF.createOrReplaceTempView("locations")

In [30]:
val poupularZones = spark.sql("""
    SELECT t.PULocationID, l.Zone, count(*) as trip_count
    FROM trips t JOIN locations l ON t.PULocationID = l.LocationID
    GROUP BY t.PULocationID, l.Zone
    ORDER BY trip_count DESC
    LIMIT 10
""")
println("Самые популярные районы для заказов")
poupularZones.show(false)

Самые популярные районы для заказов
+------------+----------------------------+----------+
|PULocationID|Zone                        |trip_count|
+------------+----------------------------+----------+
|237         |Upper East Side South       |15945     |
|161         |Midtown Center              |15255     |
|236         |Upper East Side North       |13767     |
|162         |Midtown East                |13715     |
|170         |Murray Hill                 |11702     |
|234         |Union Sq                    |11488     |
|230         |Times Sq/Theatre District   |11455     |
|186         |Penn Station/Madison Sq West|10319     |
|48          |Clinton East                |10091     |
|163         |Midtown North               |9845      |
+------------+----------------------------+----------+



poupularZones: DataFrame = [PULocationID: int, Zone: string ... 1 more field]

In [31]:
poupularZones.write.parquet("poupularZones.parquet")

In [31]:
// var df_top = (
//     df1.groupBy("PULocationID")
//     .count()
//     .orderBy($"count".desc)
//     .limit(1)
//     .join(df2, $"PULocationID" === $"LocationID", "inner")
//     .drop("service_zone")
// )
// df_top.show(10, false)

In [31]:
// df_top.createOrReplaceTempView("table1")

In [31]:
// val res = spark.sql("SELECT * FROM table1")

In [32]:
val tripsRDD = tripsDF.rdd

tripsRDD: org.apache.spark.rdd.RDD[Row] = MapPartitionsRDD[136] at rdd at cmd32.sc:1

In [45]:
val popularTime = tripsRDD.map(
    x => x.getTimestamp(x.fieldIndex("tpep_pickup_datetime")).getHours()
).map(word => (word,1)).reduceByKey(_+_).sortBy(_._2, false)
val res = popularTime.collect()
println("Статистика по времени (час), когда происходит больше всего вызовов")
res.foreach(println)

cmd45.sc:2: method getHours in class Date is deprecated
    x => x.getTimestamp(x.fieldIndex("tpep_pickup_datetime")).getHours()
                                                              ^


Статистика по времени (час), когда происходит больше всего вызовов
(19,22121)
(20,21598)
(22,20884)
(21,20318)
(23,19528)
(9,18867)
(18,18664)
(16,17843)
(15,17483)
(10,16840)
(17,16160)
(14,16082)
(13,16001)
(12,15564)
(8,15445)
(11,15348)
(0,14652)
(7,8600)
(1,7050)
(2,3978)
(6,3133)
(3,2538)
(4,1610)
(5,1586)


popularTime: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[224] at sortBy at cmd45.sc:3
res: Array[(Int, Int)] = Array(
  (19, 22121),
  (20, 21598),
  (22, 20884),
  (21, 20318),
  (23, 19528),
  (9, 18867),
  (18, 18664),
  (16, 17843),
  (15, 17483),
  (10, 16840),
  (17, 16160),
  (14, 16082),
  (13, 16001),
  (12, 15564),
  (8, 15445),
  (11, 15348),
  (0, 14652),
  (7, 8600),
  (1, 7050),
  (2, 3978),
  (6, 3133),
  (3, 2538),
  (4, 1610),
  (5, 1586)
)

In [48]:
val popularTimeDF = spark.createDataFrame(popularTime)
popularTimeDF.write.parquet("popularTime.parquet")

popularTimeDF: DataFrame = [_1: int, _2: int]

In [50]:
// https://stackoverflow.com/questions/44516627/how-to-convert-a-dataframe-to-dataset-in-apache-spark-in-scala

case class Trip(VendorID: Int)

val encoder = org.apache.spark.sql.Encoders.product[Trip]
val tripsDS = tripsDF.as(encoder)
tripsDS.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2| 2018-01-25 02:02:56|  2018-01-25 02:10:58|              1|         2.02|         1|                 N|          48|         107|           2|        8.5|  0.5|    0.5|       0.0|         0.0|                  0.3|         9.8|
|       2| 2018-01-25 02:57:13|  2018-01-25 03:2

defined class Trip
encoder: Encoder[Trip] = ExpressionEncoder(
  objSerializer = If(
    predicate = IsNull(
      child = BoundReference(
        ordinal = 0,
        dataType = ObjectType(cls = class ammonite.$sess.cmd50$Helper$Trip),
        nullable = true
      )
    ),
    trueValue = Literal(
      value = null,
      dataType = Seq(
        StructField(
          name = "VendorID",
          dataType = IntegerType,
          nullable = false,
          metadata = {}
        )
      )
    ),
    falseValue = CreateNamedStruct(
      children = List(
        Literal(value = VendorID, dataType = StringType),
        Invoke(
          targetObject = KnownNotNull(
            child = BoundReference(
              ordinal = 0,
              dataType = ObjectType(
                cls = class ammonite.$sess.cmd50$Helper$Trip
              ),
              nullable = true
            )
          ),
          functionName = "VendorID",
          dataType = IntegerType,
          argument

In [51]:
val tripsDSAvg: Double = tripsDS.agg(avg(tripsDS.col("trip_distance"))).collect()(0).getDouble(0)
val tripsExtDS = (tripsDS
     .withColumn("dstAvgSub", tripsDS.col("trip_distance") - tripsDSAvg)
     .withColumn("dstAvgSubSqr", $"dstAvgSub" * $"dstAvgSub")
)
tripsExtDS.createOrReplaceTempView("trips_ext")
tripsExtDS.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|           dstAvgSub|        dstAvgSubSqr|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+
|       2| 2018-01-25 02:02:56|  2018-01-25 02:10:58|              1|         2.02|         1|                 N|          48|         107|           2|        

tripsDSAvg: Double = 2.717989442380494
tripsExtDS: DataFrame = [VendorID: int, tpep_pickup_datetime: timestamp ... 17 more fields]

In [55]:
val res = spark.sql("""
    SELECT
        count(*),
        min(trip_distance),
        max(trip_distance),
        avg(trip_distance),
        sqrt( (1/count(*)) * sum(dstAvgSubSqr)) as std_dev
    FROM trips_ext
""")
println("Статистика по поездкам")
res.show(false)
res.write.parquet("tripsStat.parquet")

Статистика по поездкам
+--------+------------------+------------------+------------------+-----------------+
|count(1)|min(trip_distance)|max(trip_distance)|avg(trip_distance)|std_dev          |
+--------+------------------+------------------+------------------+-----------------+
|331893  |0.0               |66.0              |2.717989442380494 |3.485146974465618|
+--------+------------------+------------------+------------------+-----------------+



res: DataFrame = [count(1): bigint, min(trip_distance): double ... 3 more fields]